Dans ce mini projet, il est demandé de concevoir un système de recherche d'information en implémentant deux modèles de base:


*   Modèle vectoriel
*   Modèle booléen


Puis, effectuer les expérimentations nécessaires afin de fixer le seuil et la taille de réponse


Vous pouvez trouver a la fin une section pour tester les modèles. Cependant nous avons établie une interface pour le teste en générale


---




Redirectionner vers le dossier du projet RI dans le drive

In [217]:
%cd /content/drive/MyDrive/RI

/content/drive/MyDrive/RI


In [218]:
%ls

Application/  Documents  qrels.text  RI.ipynb
cacm.all      pickle/    query.text  ri.py


Les imports nécessaires pour le bon fonctionnement de notre mini-projet

In [187]:
import re
import pickle
import time
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
import os
from collections import OrderedDict 

In [188]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Extraction des documents

Lecture de la collection

---



In [189]:
#Ouverture de cacm.all
c=open("cacm.all","r")
var=c.readlines() 

Extraction des documents des collections

---



In [190]:
start_time = time.time()  

Liste_Textes=[] 
#Extraire les blocs de documents à partir de .I ..... jusqu'à ...... .X
for i in var:
	if i.startswith('.I'):
		Texte=''
	Texte=Texte+str(i)
	if i.startswith('.X'):
		Liste_Textes.append(Texte)

Documents={}
for texte_doc in Liste_Textes:
	y=re.search(".I\s(\d*)\n.T((.|\n)*?)(.W\n((.*\n)*))?(.B(.|\n)*?)(.A\n((.*\n)*))?.N",str(texte_doc))
	if y:
		Documents[str(y.group(1))]=str(y.group(2))
		if(y.group(5)):
			Documents[str(y.group(1))]=Documents[str(y.group(1))]+str(y.group(5))
		if(y.group(10)):
			Documents[str(y.group(1))]=Documents[str(y.group(1))]+str(y.group(10))

interval = time.time() - start_time  
print ("Temps d'extraction  %.02f" % (interval)," secondes"  )

Temps d'extraction  0.14  secondes


Sauvegarder les documents dans un fichier pickle

---



In [ ]:
#Stocker les documents dans un fichier pickle
outfile = open("./pickle/Documents",'wb')
pickle.dump(Documents,outfile)

In [ ]:
file_stats = os.stat("./pickle/Documents")
print(f'Taille documents en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille documents en MegaBytes est 1.2424001693725586


#Indexation

Récupérer les documents de la collection

---



In [191]:
#ouverture du fichier contenant les documents
infile = open("./pickle/Documents",'rb')
Documents= pickle.load(infile)      #Documents est un dictio clé: num du document ------> Valeur: titre+ résumé+ Auteur

Visualiser les documents de la collections

---



In [ ]:
Documents

{'1': '\nPreliminary Report-International Algebraic Language\nPerlis, A. J.\nSamelson,K.\n',
 '2': '\nExtraction of Roots by Repeated Subtractions for Digital Computers\nSugai, I.\n',
 '3': '\nTechniques Department on Matrix Program Schemes\nFriedman, M. D.\n',
 '4': '\nGlossary of Computer Engineering and Programming Terminology\n',
 '5': '\nTwo Square-Root Approximations\n',
 '6': '\nThe Use of Computers in Inspection Procedures\n',
 '7': '\nGlossary of Computer Engineering and Programming Terminology\n',
 '8': '\nOn The Equivalence and Transformation of Program Schemes\nFriedman, M. D.\n',
 '9': '\nProposal for an UNCOL\nConway, M. E.\n',
 '10': '\nGlossary of Computer Engineering and Programming Terminology\n',
 '11': '\nThe Problem of Programming Communication with\nChanging Machines A Proposed Solution-Part 2\nStrong, J.\nWegstein, J.\nTritter, A.\nOlsztyn, J.\nMock, O.\nSteel, T.\n',
 '12': '\nError Estimation in Runge-Kutta Procedures\nCall, D. H.\nReeves, R. F.\n',
 '13': '\nG

##fichier indexe

La fonction traiter_document est une fonction qui permet d'effectuer un prétraitement sur les documents


---



In [192]:
def traiter_document(var):
	#remplacer les '
	var=var.replace(","," ")
	var=var.replace("."," ")
	var=var.replace("-"," ")
	var=var.replace("!"," ")
	var=var.replace("?"," ")
	var=var.replace(":"," ")
	var=var.replace("="," ")
	var=var.replace("@"," ")
	var=var.replace("&"," ")
	var=var.replace("$"," ")
	var=var.replace("("," ")
	var=var.replace(")"," ")
	var=var.replace("["," ")
	var=var.replace("]"," ")
	var=var.replace("{"," ")
	var=var.replace("}"," ")
	var=var.replace("<"," ")
	var=var.replace(">"," ")
	var=var.replace("'"," ")
	var=var.replace("`"," ")
	var=var.replace("|"," ")
	ponctuation=["\"", "'", "!", "(", ")", "?", ":", "=", ",", ";", ".","`",">","<","}","{","]","[","&","@","|"]

	#decouper le texte en mots et mettre tous les mots en minuscules
	liste_des_mots=word_tokenize(var)
	#liste_des_mots = [word.lower() for word in liste_des_mots ]
	
	#supprimer les mots vides
	#print(stopwords.words('english'))
  
	liste_des_mots = [word for word in liste_des_mots if word.lower() not in stopwords.words("english")]
  

	#supprimer la ponctuation
	liste_des_mots = [word for word in liste_des_mots if word not in ponctuation]

	#supprimer les mots à une seule lettre
	liste_des_mots = [word for word in liste_des_mots if len(word)>1]

	#calcuer la frequence de chaque mot du document et mettre le tout dans un dictionnaire "dict_frequence"
	dict_frequence=dict(collections.Counter(liste_des_mots))

	return(dict_frequence)

Création du fichier index en calculant le temps d'exécution

---



In [ ]:
#dict contenant l'index
index=dict()

start_time = time.time()  
#indexer tous les documents
for cle in Documents.keys():
	var=Documents[cle]
	index[cle]=traiter_document(var)
 
interval = time.time() - start_time  
print ("Temps dde création du fichier indexe  %.02f" % (interval)," secondes"  )

Temps dde création du fichier indexe  17.66  secondes


Sauvegarder l'index dans un fichier pickle

---



In [ ]:
#Stocker le fichier index dans un fichier pickle
outfile1 = open("./pickle/index",'wb')
pickle.dump(index,outfile1)

Mesurer la taille physique du fichier index

---



In [ ]:
file_stats = os.stat("./pickle/index")
print(f'Taille index en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille index en MegaBytes est 1.7305374145507812


## fichier inverse

Création du fichier inverse

---



In [ ]:
#Fonction de creation du fichier inverse  (mot,doc) -------> frequence 
def reverse_document(index):
	reverse_dic = dict()
	for numdoc,value in index.items():
		num = numdoc
		for worddic,frequencydic in value.items():
			word = worddic
			frequency = frequencydic
			#print(num,word,frequency)
			reverse_dic[(word,num)] =frequency
	return reverse_dic

In [ ]:
#dict contenant le fichier inverse (mot,doc) -------> frequence
reverse_dic =dict()

start_time = time.time() 
reverse_dic = reverse_document(index)
interval = time.time() - start_time  
print ("Temps dde création du fichier inverse avec fréquence  %.02f" % (interval)," secondes"  )

Temps dde création du fichier inverse avec fréquence  0.04  secondes


In [ ]:
#Stocker le fichier inverse dans un fichier pickle
outfile3 = open("./pickle/Fichier_inverse_meth1",'wb')
pickle.dump(reverse_dic,outfile3)

In [ ]:
file_stats = os.stat("./pickle/Fichier_inverse_meth1")
print(f'Taille inverse en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille inverse en MegaBytes est 2.64028263092041


##fichier inverse podéré

In [ ]:
def poids(ti,dj):
    freqtidj = reverse_dic[(ti,dj)]  # ----> freq(ti,dj) 
    freqdj = sorted(list(index[dj].values()), reverse=True)[0]   # ---> max(freq(dj))
    N = 3204
    #ni
    mylist = [mot for mot in list(reverse_dic.keys()) if mot[0]==ti]
    ni = len(mylist)
    return float("{:.3f}".format((freqtidj/freqdj)*math.log10((N/ni)+1)))

In [ ]:
#Fonction de creation du fichier inverse pondere (mot,doc) ----> ponderation
def reverse_document_pondere(reverse_dic):
	reverse_dic_pondere = dict()
	for key in reverse_dic.keys():
		reverse_dic_pondere[key] = poids(key[0], key[1])
	return reverse_dic_pondere

In [ ]:
#dict contenant le fichier inverse (mot,doc) -------> pondération
reverse_dic_pondere =dict()

start_time = time.time() 
#Construction d'un fichier inverse pondéré
reverse_dic_pondere = reverse_document_pondere(reverse_dic)
interval = time.time() - start_time  
print ("Temps de création du fichier inverse pondéré  %.02f" % (interval)," secondes"  )

Temps de création du fichier inverse pondéré  412.15  secondes


In [ ]:
#Stocker le fichier inverse pondéré dans un fichier pickle
outfile4 = open("./pickle/Fichier_inverse_pondere",'wb')
pickle.dump(reverse_dic_pondere, outfile4)

In [ ]:
file_stats = os.stat("./pickle/Fichier_inverse_pondere")
print(f'Taille inverse pondéré en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille inverse pondéré en MegaBytes est 3.244595527648926


#les fonctions d'acces

In [ ]:
def Accès_par_dic(num_Doc):
	return index[num_Doc]

In [ ]:
def Accès_par_mot(mot):
	return Fichier_inverse[mot]

In [ ]:
def Accès_par_mot_doc(mot,num_Doc):
	return Fichier_inverse_meth1[(mot,num_Doc)]

#les modèles

##Modéle booléen

In [ ]:
def Modèle_Boolean(Request,Index):
    #liste qui contiendera les documents vérifiés
    Reponse_Documents=[]
    #tokenization de la requête
    Requete_tokens=nltk.tokenize.word_tokenize(Request)  
    #Requete_tokens=['(','eau','and','poisson',')','or','chèvre']
    # stocker dans une liste les termes de la requete sans les operateur 
    Liste_termes_Req=[]
    for exp in Requete_tokens:
            if(exp.lower() not in ['and','or','(',')','not']):
                Liste_termes_Req.append(exp)
   
    #Parcourir tous les documents de l'index
    for doc in Index.keys():
        #print('doc'+str(doc))
        #pour chaque document on a besoin d'une nouvelle copie des tokens de a requete
        #Requete=Requete_tokens 
        Requete=nltk.tokenize.word_tokenize(Request)  
        #Requete=['(','eau','and','poisson',')','and','not','chèvre']
        #print(Liste_termes_Req)
        for terme in Liste_termes_Req:
            #print(terme)
            if(terme in Index[doc].keys()):
                #print(Requete)
                Requete[Requete.index(terme)]='1'
                #print('oui')
            else:
                Requete[Requete.index(terme)]='0'
                #print('non')
        
        String_Requete=" ".join(Requete)
        if(eval(String_Requete))==1:
            Reponse_Documents.append(doc)
            #print(Reponse_Documents)
    return(Reponse_Documents)

In [ ]:
infile3 = open("./pickle/index",'rb')
index= pickle.load(infile3) 

infile = open("./pickle/Fichier_inverse_meth1",'rb')
reverse_dic= pickle.load(infile) 

infile = open("./pickle/Fichier_inverse_pondere",'rb')
reverse_dic_pondere= pickle.load(infile) 

Tester quelque requetes avec le modèle booléen

---



In [ ]:
Requ='(preliminary or (report and time)) and not computer'

start_time = time.time()
reponse = Modèle_Boolean(Requ,index)
print(reponse)
interval = time.time() - start_time  
print ("Temps de réponse du modele booleen  %.02f" % (interval)," secondes"  )

['254', '637', '894', '1349', '2380', '2556', '2970']
Temps de réponse du modele booleen  0.36  secondes


In [ ]:
Requ='life and test or ( not open and books )'

start_time = time.time()
reponse = Modèle_Boolean(Requ,index)
print(reponse)
interval = time.time() - start_time  
print ("Temps de réponse du modele booleen  %.02f" % (interval)," secondes"  )

['397']
Temps de réponse du modele booleen  0.30  secondes


In [ ]:
Requ="(science or compiler) and not algebra and code"

start_time = time.time()
reponse = Modèle_Boolean(Requ,index)
print(reponse)
interval = time.time() - start_time  
print ("Temps de réponse du modele booleen  %.02f" % (interval)," secondes"  )

['123', '1223', '1551', '1613', '1665', '1807', '2423', '2433', '2897', '3080']
Temps de réponse du modele booleen  0.34  secondes


In [ ]:
Requ="(extraction and roots) or use"

start_time = time.time()
reponse = Modèle_Boolean(Requ,index)
print(reponse)
interval = time.time() - start_time  
print ("Temps de réponse du modele booleen  %.02f" % (interval)," secondes"  )

['48', '78', '95', '254', '265', '272', '321', '329', '396', '409', '440', '441', '462', '464', '531', '616', '679', '699', '719', '759', '828', '855', '858', '894', '940', '952', '972', '980', '990', '1028', '1051', '1062', '1098', '1122', '1153', '1165', '1173', '1179', '1183', '1188', '1210', '1213', '1216', '1223', '1233', '1234', '1235', '1258', '1262', '1272', '1295', '1307', '1308', '1327', '1341', '1358', '1359', '1362', '1365', '1381', '1389', '1400', '1402', '1408', '1411', '1453', '1456', '1467', '1482', '1483', '1501', '1505', '1506', '1515', '1519', '1523', '1534', '1536', '1543', '1563', '1572', '1589', '1595', '1601', '1608', '1620', '1623', '1626', '1627', '1637', '1646', '1651', '1652', '1665', '1676', '1677', '1684', '1695', '1699', '1708', '1722', '1725', '1735', '1738', '1745', '1750', '1753', '1767', '1769', '1784', '1808', '1810', '1824', '1854', '1861', '1870', '1879', '1887', '1925', '1931', '1932', '1948', '1957', '1958', '1975', '1976', '1977', '1978', '1989',

##modele vectoriel

Récupérer les 64 requêtes 

---



In [ ]:
c2=open("query.text","r")
var2=c2.readlines()
print(var2)

['.I 1\n', '.W\n', ' What articles exist which deal with TSS (Time Sharing System), an\n', 'operating system for IBM computers?\n', '.N\n', ' 1. Richard Alexander, Comp Serv, Langmuir Lab (TSS)\n', ' \n', '.I 2\n', '.W\n', ' I am interested in articles written either by Prieve or Udo Pooch\n', '.A\n', 'Prieve, B.\n', 'Pooch, U.\n', '.N\n', ' 2. Richard Alexander, Comp Serv, Langmuir Lab (author = Pooch or Prieve)\n', ' \n', '.I 3\n', '.W\n', ' Intermediate languages used in construction of multi-targeted compilers; TCOLL\n', '.N\n', ' 3. Donna Bergmark, Comp Serv, Uris Hall (intermed lang)\n', ' \n', '.I 4\n', '.W\n', " I'm interested in mechanisms for communicating between disjoint processes,\n", 'possibly, but not exclusively, in a distributed environment.  I would\n', 'rather see descriptions of complete mechanisms, with or without implementations,\n', 'as opposed to theoretical work on the abstract problem.  Remote procedure\n', 'calls and message-passing are examples of my interes

Création d'un dictionnaire contenant les requêtes

In [ ]:
#créer un dictionnaire de requete
Liste_Textes=[] 
#Extraire les blocs de documents à partir de .I ..... jusqu'à ...... .X
for i in var2:
  if i.startswith('.I'):
    Liste_Textes.append(Texte)
    Texte=''
  Texte=Texte+str(i)
Documents={}
for texte_doc in Liste_Textes:
	y=re.search(".I\s(\d*)\n(.T((.|\n)*?))?(.W\n((.*\n)*))?(.A\n((.*\n)*))?(.N\n((.*\n)*))",str(texte_doc))
	if y:
		Documents[str(y.group(1))]=	''
		if (y.group(6)):
			Documents[str(y.group(1))]=Documents[str(y.group(1))]+str(y.group(6))
		if (y.group(12)):
			Documents[str(y.group(1))]=Documents[str(y.group(1))]+str(y.group(12))
		if (y.group(2)):
			Documents[str(y.group(1))]=Documents[str(y.group(1))]+str(y.group(2))

Indexer les requêtes

---



In [ ]:
#index requeste
indexReq=dict()
for i in range(1,65):
  indexReq[str(i)] = traiter_document(Documents[str(i)])

visualiser l'indexe des requetes

---



In [ ]:
print(indexReq)

{'1': {'articles': 1, 'exist': 1, 'deal': 1, 'TSS': 2, 'Time': 1, 'Sharing': 1, 'System': 1, 'operating': 1, 'system': 1, 'IBM': 1, 'computers': 1, 'Richard': 1, 'Alexander': 1, 'Comp': 1, 'Serv': 1, 'Langmuir': 1, 'Lab': 1}, '2': {'interested': 1, 'articles': 1, 'written': 1, 'either': 1, 'Prieve': 3, 'Udo': 1, 'Pooch': 3, 'Richard': 1, 'Alexander': 1, 'Comp': 1, 'Serv': 1, 'Langmuir': 1, 'Lab': 1, 'author': 1}, '3': {'Intermediate': 1, 'languages': 1, 'used': 1, 'construction': 1, 'multi': 1, 'targeted': 1, 'compilers': 1, 'TCOLL': 1, 'Donna': 1, 'Bergmark': 1, 'Comp': 1, 'Serv': 1, 'Uris': 1, 'Hall': 1, 'intermed': 1, 'lang': 1}, '4': {'interested': 1, 'mechanisms': 2, 'communicating': 1, 'disjoint': 2, 'processes': 2, 'possibly': 1, 'exclusively': 1, 'distributed': 1, 'environment': 1, 'would': 1, 'rather': 1, 'see': 1, 'descriptions': 1, 'complete': 1, 'without': 1, 'implementations': 1, 'opposed': 1, 'theoretical': 1, 'work': 1, 'abstract': 1, 'problem': 1, 'Remote': 1, 'procedur

préatratement sur les requetes

---



In [ ]:
def cleanReq(var):
    var=var.replace(","," ")
    var=var.replace("."," ")
    var=var.replace("-"," ")
    var=var.replace("!"," ")
    var=var.replace("?"," ")
    var=var.replace(":"," ")
    var=var.replace("="," ")
    var=var.replace("@"," ")
    var=var.replace("&"," ")
    var=var.replace("$"," ")
    var=var.replace("("," ")
    var=var.replace(")"," ")
    var=var.replace("["," ")
    var=var.replace("]"," ")
    var=var.replace("{"," ")
    var=var.replace("}"," ")
    var=var.replace("<"," ")
    var=var.replace(">"," ")
    var=var.replace("'"," ")
    var=var.replace("`"," ")
    var=var.replace("|"," ")
    #var  = var.lower()
    return var

Representation des requetes 

---



In [ ]:
def codeReq(var,num):
    dicReq = dict()
    index = indexReq[num]
    for term in index:
      freq = index[term]
      maximum = sorted(list(index.values()))[0]
      dicReq[term] = float("{:.3f}".format(freq/maximum))
    return dicReq

Les mesures de similarités

---



In [ ]:
def mesureInterne(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        somme = 0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme)))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

In [ ]:
def mesureDice(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format(((2*somme) / (sommedoc+sommereq))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

In [ ]:
def mesureCosinus(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme / math.sqrt(sommedoc*sommereq))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

In [ ]:
def mesureJacard(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme / (sommedoc+sommereq-somme))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

In [ ]:
def Modèle_Vectoriel(req,reverse_dic_pondere,nbDocument):
    print("1- Methode de produit Interne.")
    print("2- Methode de Coefficient de Dice.")
    print("3- Methode de Mesure du Cosinus.")
    print("4- Methode de Mesure du Jaccard.")
    print("Veuillez choisiri votre methode: ")
    methode = int(input())
    if methode == 1:
        print(mesureInterne(req,reverse_dic_pondere,nbDocument))
    elif methode == 2:
        print(mesureDice(req,reverse_dic_pondere,nbDocument))
    elif methode == 3:
        print(mesureCosinus(req,reverse_dic_pondere,nbDocument))
    elif methode == 4:
        print(mesureJacard(req,reverse_dic_pondere,nbDocument))
    else:
        print("Numero incorrecte")

Mesurer la similarité avec les requêtes collectés précédemment

---



In [ ]:
resultat = dict()
for num in range(1,65):
  dic_req = codeReq(cleanReq(Documents[str(num)]), str(num))
  nbDoc = 3204
  #recuperer les clés du fichier inverse sous forme de liste
  reversed_dic_list = [mot for mot in list(reverse_dic_pondere.keys())]

  print("Requête %d" %num)
  #LE calcul mesure Interne
  start_time = time.time() 
  interne = mesureInterne(dic_req,reverse_dic_pondere,nbDoc)
  interval = time.time() - start_time  
  print ("Temps d'execution de la mesure Interne est  %.02f" % (interval)," secondes"  )
  resultat[(num,'interne')] = (interne, interval)

  #Calcul mesure Dice
  start_time = time.time() 
  dice = mesureDice(dic_req,reverse_dic_pondere,nbDoc)
  interval = time.time() - start_time  
  print ("Temps d'execution de la mesure dice est  %.02f" % (interval)," secondes"  )
  resultat[(num,'dice')] = (dice, interval)

  #Calcul mesure cosinus
  start_time = time.time() 
  cosinus = mesureCosinus(dic_req,reverse_dic_pondere,nbDoc)
  interval = time.time() - start_time  
  print ("Temps d'execution de la mesure cosinus est  %.02f" % (interval)," secondes"  )
  resultat[(num,'cosinus')] = (cosinus, interval)

  #calcul mesure jacard
  start_time = time.time() 
  jacard = mesureJacard(dic_req,reverse_dic_pondere,nbDoc)
  interval = time.time() - start_time  
  print ("Temps d'execution de la mesure jacard est  %.02f" % (interval)," secondes"  )
  resultat[(num,'jacard')] = (jacard, interval)
  print()


Requête 1
Temps d'execution de la mesure Interne est  0.02  secondes
Temps d'execution de la mesure dice est  0.02  secondes
Temps d'execution de la mesure cosinus est  0.02  secondes
Temps d'execution de la mesure jacard est  0.02  secondes

Requête 2
Temps d'execution de la mesure Interne est  0.01  secondes
Temps d'execution de la mesure dice est  0.01  secondes
Temps d'execution de la mesure cosinus est  0.01  secondes
Temps d'execution de la mesure jacard est  0.01  secondes

Requête 3
Temps d'execution de la mesure Interne est  0.01  secondes
Temps d'execution de la mesure dice est  0.01  secondes
Temps d'execution de la mesure cosinus est  0.01  secondes
Temps d'execution de la mesure jacard est  0.02  secondes

Requête 4
Temps d'execution de la mesure Interne est  0.03  secondes
Temps d'execution de la mesure dice est  0.03  secondes
Temps d'execution de la mesure cosinus est  0.03  secondes
Temps d'execution de la mesure jacard est  0.03  secondes

Requête 5
Temps d'execution 

L'affichage des documents selectionné en utilisant la mesure interne

---



In [ ]:
for num in range(1,65):
  print("La liste des documents séléctionné pour la requête %d " %num ," est")
  print("En utilisant la mesure interne: ")
  print(resultat[(num,'interne')][0])
  print()

La liste des documents séléctionné pour la requête 1   est
En utilisant la mesure interne: 
{'1410': 5.566, '1506': 4.309, '1605': 3.719, '252': 3.253, '2597': 3.072, '289': 3.029, '325': 3.029, '339': 3.029, '271': 2.959, '1236': 2.932, '1827': 2.889, '2319': 2.879, '2439': 2.789, '1680': 2.713, '2317': 2.632, '1657': 2.559, '1750': 2.555, '103': 2.525, '1519': 2.518, '1195': 2.517, '1472': 2.413, '2878': 2.413, '2632': 2.358, '2868': 2.358, '2629': 2.334, '1752': 2.23, '2218': 2.23, '1523': 2.188, '3127': 2.178, '1844': 2.163, '1647': 2.162, '1591': 2.107, '3068': 2.103, '2541': 2.084, '3048': 2.068, '1960': 2.062, '2920': 2.042, '1751': 1.948, '1544': 1.938, '2380': 1.938, '1534': 1.863, '2379': 1.863, '1462': 1.854, '1315': 1.828, '2357': 1.82, '1391': 1.791, '2371': 1.778, '1214': 1.722, '1033': 1.684, '3025': 1.671, '1571': 1.669, '254': 1.664, '1153': 1.664, '2303': 1.664, '2312': 1.664, '2367': 1.664, '2607': 1.664, '2685': 1.664, '2557': 1.656, '73': 1.641, '174': 1.641, '203'

L'affichage des documents selectionné en utilisant la mesure dice

---



In [ ]:
for num in range(1,65):
  print("La liste des documents séléctionné pour la requête %d " %num ," est")
  print("En utilisant la mesure dice: ")
  print(resultat[(num,'dice')][0])
  print()

La liste des documents séléctionné pour la requête 1   est
En utilisant la mesure dice: 
{'1381': 1.0, '2320': 1.0, '2777': 1.0, '39': 0.997, '242': 0.997, '617': 0.997, '728': 0.997, '731': 0.997, '980': 0.997, '1034': 0.997, '1087': 0.997, '1108': 0.997, '1170': 0.997, '1307': 0.997, '1325': 0.997, '1332': 0.997, '1484': 0.997, '1502': 0.997, '1665': 0.997, '2126': 0.997, '2188': 0.997, '2221': 0.997, '2306': 0.997, '2390': 0.997, '2435': 0.997, '2581': 0.997, '2618': 0.997, '2690': 0.997, '2741': 0.997, '2747': 0.997, '2768': 0.997, '2841': 0.997, '2845': 0.997, '3089': 0.997, '3144': 0.997, '3153': 0.997, '3174': 0.997, '2920': 0.996, '68': 0.995, '2357': 0.995, '1391': 0.993, '2439': 0.993, '2209': 0.992, '1164': 0.991, '96': 0.989, '558': 0.989, '827': 0.989, '1145': 0.989, '1166': 0.989, '1260': 0.989, '1622': 0.989, '3201': 0.989, '265': 0.986, '759': 0.986, '251': 0.983, '894': 0.983, '1167': 0.983, '1808': 0.983, '1929': 0.983, '2147': 0.983, '2163': 0.983, '2304': 0.983, '23

L'affichage des documents selectionné en utilisant la mesure cosinus


---



In [ ]:
for num in range(1,65):
  print("La liste des documents séléctionné pour la requête %d " %num ," est")
  print("En utilisant la mesure cosinus: ")
  print(resultat[(num,'cosinus')][0])
  print()

La liste des documents séléctionné pour la requête 1   est
En utilisant la mesure cosinus: 
{'39': 1.0, '60': 1.0, '68': 1.0, '73': 1.0, '78': 1.0, '96': 1.0, '174': 1.0, '190': 1.0, '197': 1.0, '202': 1.0, '203': 1.0, '242': 1.0, '243': 1.0, '251': 1.0, '254': 1.0, '265': 1.0, '275': 1.0, '276': 1.0, '277': 1.0, '282': 1.0, '284': 1.0, '289': 1.0, '294': 1.0, '325': 1.0, '334': 1.0, '339': 1.0, '367': 1.0, '396': 1.0, '398': 1.0, '439': 1.0, '558': 1.0, '586': 1.0, '594': 1.0, '617': 1.0, '619': 1.0, '656': 1.0, '675': 1.0, '704': 1.0, '705': 1.0, '719': 1.0, '728': 1.0, '731': 1.0, '759': 1.0, '767': 1.0, '768': 1.0, '798': 1.0, '822': 1.0, '825': 1.0, '826': 1.0, '827': 1.0, '851': 1.0, '852': 1.0, '854': 1.0, '856': 1.0, '887': 1.0, '888': 1.0, '891': 1.0, '894': 1.0, '942': 1.0, '950': 1.0, '963': 1.0, '964': 1.0, '973': 1.0, '980': 1.0, '1002': 1.0, '1010': 1.0, '1012': 1.0, '1030': 1.0, '1032': 1.0, '1034': 1.0, '1043': 1.0, '1066': 1.0, '1068': 1.0, '1070': 1.0, '1087': 1.0, '1

L'affichage des documents selectionné en utilisant la mesure jaccard


---



In [ ]:
for num in range(1,65):
  print("La liste des documents séléctionné pour la requête %d " %num ," est")
  print("En utilisant la mesure jacard: ")
  print(resultat[(num,'jacard')][0])
  print()

La liste des documents séléctionné pour la requête 1   est
En utilisant la mesure jacard: 
{'1381': 1.0, '2320': 1.0, '2777': 1.0, '39': 0.995, '242': 0.995, '617': 0.995, '728': 0.995, '731': 0.995, '980': 0.995, '1034': 0.995, '1087': 0.995, '1108': 0.995, '1170': 0.995, '1307': 0.995, '1325': 0.995, '1332': 0.995, '1484': 0.995, '1502': 0.995, '1665': 0.995, '2126': 0.995, '2188': 0.995, '2221': 0.995, '2306': 0.995, '2390': 0.995, '2435': 0.995, '2581': 0.995, '2618': 0.995, '2690': 0.995, '2741': 0.995, '2747': 0.995, '2768': 0.995, '2841': 0.995, '2845': 0.995, '3089': 0.995, '3144': 0.995, '3153': 0.995, '3174': 0.995, '2920': 0.992, '2357': 0.991, '68': 0.989, '1391': 0.987, '2439': 0.986, '2209': 0.983, '1164': 0.981, '96': 0.978, '558': 0.978, '827': 0.978, '1145': 0.978, '1166': 0.978, '1260': 0.978, '1622': 0.978, '3201': 0.978, '265': 0.973, '759': 0.973, '251': 0.967, '894': 0.967, '1167': 0.967, '1808': 0.967, '1929': 0.967, '2147': 0.967, '2163': 0.967, '2304': 0.967, '

Sauvegarder les résultats dans un fichier pickle

In [ ]:
#Stocker le fichier inverse pondéré dans un fichier pickle
outfile4 = open("./pickle/ResultatGlobal",'wb')
pickle.dump(resultat, outfile4)

In [ ]:
file_stats = os.stat("./pickle/ResultatGlobal")
print(f'Taille inverse pondéré en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille inverse pondéré en MegaBytes est 3.213804244995117


Mesurer le temps d'execution pour la mesure de cosinus

---



In [ ]:
starttimeGlobal = time.time()
resultatCosinus = dict()
for num in range(1,65):
  print("Requete %d"%num)
  dic_req = codeReq(cleanReq(Documents[str(num)]), str(num))
  nbDoc = 3204
  #recuperer les clés du fichier inverse sous forme de liste
  reversed_dic_list = [mot for mot in list(reverse_dic_pondere.keys())]


  start_time = time.time() 
  cosinus = mesureCosinus(dic_req,reverse_dic_pondere,nbDoc)
  interval = time.time() - start_time  
  print ("Temps d'execution de la mesure cosinus est  %.02f" % (interval)," secondes"  )
  resultatCosinus[num] = (cosinus, interval)
  print()
endtimeGlobal = time.time() - starttimeGlobal
print("Temps de réponse des 64 requêtes est : ", 0.0377)

Requete 1
Temps d'execution de la mesure cosinus est  0.02  secondes

Requete 2
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 3
Temps d'execution de la mesure cosinus est  0.02  secondes

Requete 4
Temps d'execution de la mesure cosinus est  0.03  secondes

Requete 5
Temps d'execution de la mesure cosinus est  0.02  secondes

Requete 6
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 7
Temps d'execution de la mesure cosinus est  0.02  secondes

Requete 8
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 9
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 10
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 11
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 12
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 13
Temps d'execution de la mesure cosinus est  0.01  secondes

Requete 14
Temps d'execution de la mesure cosinus est  0.02  secondes

Requete 15
Temp

Sauvegarder les résultats de similarités obtenus avec la mesures cosinus

---



In [ ]:
#Stocker le fichier inverse pondéré dans un fichier pickle
outfile4 = open("./pickle/ResultatCosinus",'wb')
pickle.dump(resultatCosinus, outfile4)

In [ ]:
file_stats = os.stat("./pickle/ResultatCosinus")
print(f'Taille inverse pondéré en MegaBytes est {file_stats.st_size / (1024 * 1024)}')

Taille inverse pondéré en MegaBytes est 0.8022727966308594


#Evaluation des requetes

récupération des requêtes pertinente

---



In [ ]:
req_file = open("qrels.text",'r')
var=req_file.readlines() 
liste_des_mots = []
for line in var:
  liste_des_mots.append(word_tokenize(line))
req_dic = dict()
for liste in liste_des_mots:
  
  if(liste[0].startswith('0')):
    if liste[0][1] not in req_dic:
      req_dic[liste[0][1]] = []
    req_dic[liste[0][1]].append(liste[1])
  else:
    if liste[0] not in req_dic:
      req_dic[liste[0]] = []
    req_dic[liste[0]].append(liste[1])

In [ ]:
cosinus = open("./pickle/ResultatCosinus",'rb')
Cos = pickle.load(cosinus)

##Rappel

In [ ]:
def Recall(requette, doc_list):
  #Le nombre de documents partinant en total
  nbDocPartinantTotal = len(requette)
  #Le nombre de documents partiant selectionne
  nbDocPartinantSelectionne = 0
  for doc in doc_list:
    if doc in requette:
      nbDocPartinantSelectionne = nbDocPartinantSelectionne + 1
  recall = nbDocPartinantSelectionne / nbDocPartinantTotal
  return recall

In [ ]:
starttimerecall = time.time()
recall_result = dict()
for key in req_dic.keys():
  doc_list = []
  for tupe in Cos[(int(key))][0]:
    doc_list.append(tupe)
  #La requette
  requette = req_dic[key]
  #Calculer le rappel pour chaque requête
  recall_result[key] = Recall(requette, doc_list)  
  print("Le rappel de la requêtte %s "%key, "est %f"% recall_result[key])
endtimerecall = time.time() - starttimerecall
print("Le temps de mesure de rappel est ",endtimerecall )

Le rappel de la requêtte 1  est 0.800000
Le rappel de la requêtte 2  est 1.000000
Le rappel de la requêtte 3  est 0.333333
Le rappel de la requêtte 4  est 0.833333
Le rappel de la requêtte 5  est 0.875000
Le rappel de la requêtte 6  est 1.000000
Le rappel de la requêtte 7  est 0.785714
Le rappel de la requêtte 8  est 1.000000
Le rappel de la requêtte 9  est 1.000000
Le rappel de la requêtte 10  est 0.628571
Le rappel de la requêtte 11  est 0.736842
Le rappel de la requêtte 12  est 0.800000
Le rappel de la requêtte 13  est 0.727273
Le rappel de la requêtte 14  est 0.272727
Le rappel de la requêtte 15  est 0.600000
Le rappel de la requêtte 16  est 0.823529
Le rappel de la requêtte 17  est 0.500000
Le rappel de la requêtte 18  est 0.727273
Le rappel de la requêtte 19  est 0.636364
Le rappel de la requêtte 20  est 1.000000
Le rappel de la requêtte 21  est 0.909091
Le rappel de la requêtte 22  est 0.647059
Le rappel de la requêtte 23  est 1.000000
Le rappel de la requêtte 24  est 0.230769
L

In [ ]:
outfile4 = open("./pickle/recallResult",'wb')
pickle.dump(recall_result, outfile4)

##Precision

In [ ]:
def Precision(requette, doc_list):
  #Le nombre de documents partinant en total
  nbDocPartinantTotal = len(doc_list)
  #Le nombre de documents partiant selectionne
  nbDocPartinantSelectionne = 0
  for doc in doc_list:
    if doc in requette:
      nbDocPartinantSelectionne = nbDocPartinantSelectionne + 1
  precision = nbDocPartinantSelectionne / nbDocPartinantTotal
  return precision

In [ ]:
starttimeprecision = time.time()
precision_result = dict()
for key in req_dic.keys():
  doc_list = []
  for tupe in Cos[(int(key))][0]:
    doc_list.append(tupe)
  #La requette
  requette = req_dic[key]
  #Calculer le rappel pour chaque requête
  precision_result[key] = Precision(requette, doc_list)
  print("La precision de la requêtte %s"%key, " est %f"%precision_result[key]) 
endtimeprecision = time.time() - starttimeprecision
print("Le temps de calcul de la precision est ", endtimeprecision)

La precision de la requêtte 1  est 0.006192
La precision de la requêtte 2  est 0.022059
La precision de la requêtte 3  est 0.004115
La precision de la requêtte 4  est 0.014025
La precision de la requêtte 5  est 0.013645
La precision de la requêtte 6  est 0.024194
La precision de la requêtte 7  est 0.029730
La precision de la requêtte 8  est 0.006410
La precision de la requêtte 9  est 0.020179
La precision de la requêtte 10  est 0.068323
La precision de la requêtte 11  est 0.056452
La precision de la requêtte 12  est 0.007663
La precision de la requêtte 13  est 0.023188
La precision de la requêtte 14  est 0.026549
La precision de la requêtte 15  est 0.023077
La precision de la requêtte 16  est 0.020927
La precision de la requêtte 17  est 0.032653
La precision de la requêtte 18  est 0.026667
La precision de la requêtte 19  est 0.026119
La precision de la requêtte 20  est 0.008475
La precision de la requêtte 21  est 0.022371
La precision de la requêtte 22  est 0.014455
La precision de la 

In [ ]:
outfile4 = open("./pickle/precisionResult",'wb')
pickle.dump(precision_result, outfile4)

#Experimentals

In [ ]:
def mesureCosinusSeuil(dic_req,reverse_dic_pondere,nbDocument, seuilMin):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0 :
            somme = float("{:.3f}".format((somme / math.sqrt(sommedoc*sommereq))))
            if( somme > seuilMin):
              dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

## Experimentation Seuil

###Mesure

Dans cette partie, nous varions le seuilMin manuellement de 0.1 à 0.9

---



In [ ]:
seuilMin = 0.9

for num in range(1,65):
  dic_req = codeReq(cleanReq(Documents[str(num)]), str(num))
  nbDoc = 3204
  start_time = time.time() 
  cosinus = sorted(mesureCosinusSeuil(dic_req,reverse_dic_pondere,nbDoc,seuilMin).items(), key=lambda t: t[1], reverse=True)
  interval = time.time() - start_time  
  resultat[num] = (cosinus,seuilMin,interval)
recall_result = dict()
for key in req_dic.keys():
  doc_list = []
  for tupe in resultat[(int(key))][0]:
    doc_list.append(tupe[0])
  #La requette
  requette = req_dic[key]
  #Calculer le rappel pour chaque requête
  recall_result[key] = Recall(requette, doc_list)  
outfile4 = open("./pickle/recallSeuil"+str(seuilMin),'wb')
pickle.dump(recall_result, outfile4)

precision_result = dict()
for key in req_dic.keys():
  doc_list = []
  for tupe in resultat[(int(key))][0]:
    doc_list.append(tupe[0])
  #La requette
  requette = req_dic[key]
  #Calculer le rappel pour chaque requête
  precision_result[key] = Precision(requette, doc_list) 
#sauvegarder dans un fichier pickle
outfile4 = open("./pickle/precisionSeuil"+str(seuilMin),'wb')
pickle.dump(precision_result, outfile4)

###Compare

Comparer les seuil et sauvegarder les résultats (prcision rappel)

---



In [ ]:
seuilGlobal = dict()
seuilMinlist = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for key in req_dic.keys():
  for seuilMin in seuilMinlist:
    outfile4 = open("recallSeuil"+str(seuilMin),'rb')
    recall1 = pickle.load(outfile4)
    outfile4 = open("precisionSeuil"+str(seuilMin),'rb')
    precision1 = pickle.load(outfile4)
    #(key,seuil) => (recall, precision)
    print("("+ str(key)+ ","+str(seuilMin)+ ") => "+ "(" + str(recall1[str(key)])+"," + str(precision1[str(key)])+ ")")
    seuilGlobal[(key,seuilMin)] = (recall1[str(key)],precision1[str(key)])



(1,0.1) => (0.8,0.006144393241167435)
(1,0.2) => (0.8,0.004761904761904762)
(1,0.3) => (0.8,0.006144393241167435)
(1,0.4) => (0.8,0.004761904761904762)
(1,0.5) => (0.8,0.004761904761904762)
(1,0.6) => (0.8,0.004784688995215311)
(1,0.7) => (0.6,0.003676470588235294)
(1,0.8) => (0.4,0.002506265664160401)
(1,0.9) => (0.2,0.0013513513513513514)
(2,0.1) => (1.0,0.022058823529411766)
(2,0.2) => (1.0,0.021739130434782608)
(2,0.3) => (1.0,0.022058823529411766)
(2,0.4) => (1.0,0.021739130434782608)
(2,0.5) => (1.0,0.021739130434782608)
(2,0.6) => (1.0,0.021739130434782608)
(2,0.7) => (1.0,0.021739130434782608)
(2,0.8) => (1.0,0.021739130434782608)
(2,0.9) => (1.0,0.021739130434782608)
(3,0.1) => (0.3333333333333333,0.00411522633744856)
(3,0.2) => (0.3333333333333333,0.0055658627087198514)
(3,0.3) => (0.3333333333333333,0.00411522633744856)
(3,0.4) => (0.3333333333333333,0.0055658627087198514)
(3,0.5) => (0.5,0.0055658627087198514)
(3,0.6) => (0.5,0.0055658627087198514)
(3,0.7) => (0.5,0.0055658

In [ ]:
outfile4 = open("./pickle/SeuilRequetteGlobal",'wb')
pickle.dump(seuilGlobal, outfile4)

In [ ]:
outfile4 = open("./pickle/SeuilRequetteGlobal",'rb')
seuilGlobal = pickle.load(outfile4)
seuilGlobal

{('1', 0.1): (0.8, 0.006144393241167435),
 ('1', 0.2): (0.8, 0.004761904761904762),
 ('1', 0.3): (0.8, 0.006144393241167435),
 ('1', 0.4): (0.8, 0.004761904761904762),
 ('1', 0.5): (0.8, 0.004761904761904762),
 ('1', 0.6): (0.8, 0.004784688995215311),
 ('1', 0.7): (0.6, 0.003676470588235294),
 ('1', 0.8): (0.4, 0.002506265664160401),
 ('1', 0.9): (0.2, 0.0013513513513513514),
 ('10', 0.1): (0.6285714285714286, 0.06832298136645963),
 ('10', 0.2): (0.6285714285714286, 0.05962059620596206),
 ('10', 0.3): (0.6285714285714286, 0.06832298136645963),
 ('10', 0.4): (0.6285714285714286, 0.05962059620596206),
 ('10', 0.5): (0.6285714285714286, 0.05962059620596206),
 ('10', 0.6): (0.6285714285714286, 0.05962059620596206),
 ('10', 0.7): (0.6285714285714286, 0.059782608695652176),
 ('10', 0.8): (0.6285714285714286, 0.06027397260273973),
 ('10', 0.9): (0.6, 0.058011049723756904),
 ('11', 0.1): (0.7368421052631579, 0.05622489959839357),
 ('11', 0.2): (0.7368421052631579, 0.05120481927710843),
 ('11',

###Meuilleur configuration pour le seuil

In [ ]:
outfile4 = open("./pickle/SeuilRequetteGlobal",'rb')
bestseuilFile = pickle.load(outfile4)
print(bestseuilFile)

{('1', 0.1): (0.8, 0.006144393241167435), ('1', 0.2): (0.8, 0.004761904761904762), ('1', 0.3): (0.8, 0.006144393241167435), ('1', 0.4): (0.8, 0.004761904761904762), ('1', 0.5): (0.8, 0.004761904761904762), ('1', 0.6): (0.8, 0.004784688995215311), ('1', 0.7): (0.6, 0.003676470588235294), ('1', 0.8): (0.4, 0.002506265664160401), ('1', 0.9): (0.2, 0.0013513513513513514), ('2', 0.1): (1.0, 0.022058823529411766), ('2', 0.2): (1.0, 0.021739130434782608), ('2', 0.3): (1.0, 0.022058823529411766), ('2', 0.4): (1.0, 0.021739130434782608), ('2', 0.5): (1.0, 0.021739130434782608), ('2', 0.6): (1.0, 0.021739130434782608), ('2', 0.7): (1.0, 0.021739130434782608), ('2', 0.8): (1.0, 0.021739130434782608), ('2', 0.9): (1.0, 0.021739130434782608), ('3', 0.1): (0.3333333333333333, 0.00411522633744856), ('3', 0.2): (0.3333333333333333, 0.0055658627087198514), ('3', 0.3): (0.3333333333333333, 0.00411522633744856), ('3', 0.4): (0.3333333333333333, 0.0055658627087198514), ('3', 0.5): (0.5, 0.0055658627087198

In [ ]:
liste = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
bestconfig = dict()
for key in req_dic.keys():
  max = bestseuilFile[(str(key),0.1)][0]
  maxseuil = 0.1
  for seuil in liste:
    if bestseuilFile[(str(key),seuil)][0] > max:
      max = bestseuilFile[(str(key),seuil)][0]
      maxseuil = seuil
  bestconfig[int(key)] = maxseuil

In [ ]:
print(dict(sorted(bestconfig.items(), key=lambda item: item[0],reverse=True)))

{64: 0.1, 63: 0.5, 62: 0.1, 61: 0.1, 60: 0.5, 59: 0.5, 58: 0.5, 57: 0.1, 49: 0.1, 48: 0.5, 45: 0.5, 44: 0.5, 43: 0.5, 42: 0.1, 40: 0.1, 39: 0.1, 38: 0.1, 37: 0.1, 36: 0.5, 33: 0.1, 32: 0.1, 31: 0.1, 30: 0.5, 29: 0.5, 28: 0.1, 27: 0.1, 26: 0.5, 25: 0.5, 24: 0.5, 23: 0.1, 22: 0.5, 21: 0.1, 20: 0.1, 19: 0.1, 18: 0.1, 17: 0.5, 16: 0.1, 15: 0.1, 14: 0.5, 13: 0.1, 12: 0.1, 11: 0.5, 10: 0.1, 9: 0.1, 8: 0.1, 7: 0.5, 6: 0.1, 5: 0.1, 4: 0.5, 3: 0.5, 2: 0.1, 1: 0.1}


In [ ]:
outfile4 = open("./pickle/SeuilApresExperimentation",'wb')
pickle.dump(bestconfig, outfile4)

##Experiemntation taille

In [ ]:
outfile4 = open("./pickle/ResultatCosinus",'rb')
cosinuslist = pickle.load(outfile4)

In [ ]:
#calcul des precisions en variant les tailles
recall_result = dict()
precision_result = dict()
resultat_taille = dict()
for key in req_dic.keys():
  dic_req = codeReq(cleanReq(Documents[str(key)]), str(key))
  seuile = bestconfig[int(key)]
  #seuile = 0.2
  nbDoc = 3204
  requette = req_dic[key]
  for taille in range(1,len(cosinuslist[int(key)][0]),1):
    cosinus = sorted(mesureCosinusSeuil(dic_req,reverse_dic_pondere,nbDoc,seuile).items(), key=lambda t: t[1], reverse=True)[:taille]
    #resultat_taille[(key,taille)] = cosinus
    doc_list = []
    for tupe in cosinus:
      doc_list.append(tupe[0])
    
    resultat_taille[(key,taille)] = (Recall(requette, doc_list),Precision(requette, doc_list) ) 

In [ ]:
print(resultat_taille)

{('1', 1): (0.0, 0.0), ('1', 2): (0.0, 0.0), ('1', 3): (0.0, 0.0), ('1', 4): (0.0, 0.0), ('1', 5): (0.0, 0.0), ('1', 6): (0.0, 0.0), ('1', 7): (0.0, 0.0), ('1', 8): (0.0, 0.0), ('1', 9): (0.0, 0.0), ('1', 10): (0.0, 0.0), ('1', 11): (0.0, 0.0), ('1', 12): (0.0, 0.0), ('1', 13): (0.0, 0.0), ('1', 14): (0.0, 0.0), ('1', 15): (0.0, 0.0), ('1', 16): (0.0, 0.0), ('1', 17): (0.0, 0.0), ('1', 18): (0.0, 0.0), ('1', 19): (0.0, 0.0), ('1', 20): (0.0, 0.0), ('1', 21): (0.0, 0.0), ('1', 22): (0.0, 0.0), ('1', 23): (0.0, 0.0), ('1', 24): (0.0, 0.0), ('1', 25): (0.0, 0.0), ('1', 26): (0.0, 0.0), ('1', 27): (0.0, 0.0), ('1', 28): (0.0, 0.0), ('1', 29): (0.0, 0.0), ('1', 30): (0.0, 0.0), ('1', 31): (0.0, 0.0), ('1', 32): (0.0, 0.0), ('1', 33): (0.0, 0.0), ('1', 34): (0.0, 0.0), ('1', 35): (0.0, 0.0), ('1', 36): (0.0, 0.0), ('1', 37): (0.0, 0.0), ('1', 38): (0.0, 0.0), ('1', 39): (0.0, 0.0), ('1', 40): (0.0, 0.0), ('1', 41): (0.0, 0.0), ('1', 42): (0.0, 0.0), ('1', 43): (0.0, 0.0), ('1', 44): (0.0, 0.

In [ ]:
outfile4 = open("./pickle/TailleExperimentation",'wb')
pickle.dump(resultat_taille,outfile4)

###Meuilleur configuration de taille

Meuilleur cofiguration pour la taille pour chaque requette

In [ ]:
outfile4 = open("./pickle/TailleExperimentation",'rb')
tailleFile = pickle.load(outfile4)

In [ ]:
bestconfig = dict()
for key in req_dic.keys():
  max = tailleFile[(str(key),1)]
  maxtaille = 1
  for taille in range(1,len(cosinuslist[int(key)][0]),1):
    if tailleFile[(str(key),taille)] > max:
      max = tailleFile[(str(key),taille)]
      maxtaille = taille
  bestconfig[int(key)] = maxtaille

In [ ]:
print(bestconfig)

{1: 616, 2: 118, 3: 484, 4: 701, 5: 489, 6: 47, 7: 738, 8: 458, 9: 445, 10: 321, 11: 244, 12: 521, 13: 338, 14: 433, 15: 257, 16: 666, 17: 242, 18: 295, 19: 267, 20: 353, 21: 445, 22: 760, 23: 360, 24: 143, 25: 729, 26: 429, 27: 480, 28: 735, 29: 533, 30: 573, 31: 842, 32: 876, 33: 827, 36: 603, 37: 1002, 38: 776, 39: 808, 40: 702, 42: 704, 43: 718, 44: 618, 45: 904, 48: 1208, 49: 594, 57: 1, 58: 815, 59: 844, 60: 687, 61: 811, 62: 242, 63: 367, 64: 303}


In [ ]:
outfile4 = open("./pickle/TailleApresExperimentation",'wb')
pickle.dump(bestconfig, outfile4)

#Console

In [212]:
import re
import pickle
import time
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
import os
from collections import OrderedDict 


def Modèle_Boolean(Request,Index):
    #liste qui contiendera les documents vérifiés
    Reponse_Documents=[]
    #tokenization de la requête
    Requete_tokens=nltk.tokenize.word_tokenize(Request)  
    #Requete_tokens=['(','eau','and','poisson',')','or','chèvre']
    # stocker dans une liste les termes de la requete sans les operateur 
    Liste_termes_Req=[]
    for exp in Requete_tokens:
            if(exp.lower() not in ['and','or','(',')','not']):
                Liste_termes_Req.append(exp)
   
    #Parcourir tous les documents de l'index
    for doc in Index.keys():
        #print('doc'+str(doc))
        #pour chaque document on a besoin d'une nouvelle copie des tokens de a requete
        #Requete=Requete_tokens 
        Requete=nltk.tokenize.word_tokenize(Request)  
        #Requete=['(','eau','and','poisson',')','and','not','chèvre']
        #print(Liste_termes_Req)
        for terme in Liste_termes_Req:
            #print(terme)
            if(terme in Index[doc].keys()):
                #print(Requete)
                Requete[Requete.index(terme)]='1'
                #print('oui')
            else:
                Requete[Requete.index(terme)]='0'
                #print('non')
        
        String_Requete=" ".join(Requete)
        if(eval(String_Requete))==1:
            Reponse_Documents.append(doc)
            #print(Reponse_Documents)
    return(Reponse_Documents)

def Rechrcher_terme(mot):
    infile3 = open("./pickle/Fichier_inverse_meth1",'rb')
    inverse= pickle.load(infile3) 
    start_time = time.time() 
    trouver=False
    for key in inverse.keys():
      if(key[0]) == mot:
        trouver=True
        print("Document "+str(key[1])+"   ------>  Frequence d'apparirion "+str(inverse[key]))
    if trouver == False:
      print("Ce mot n'existe pas dans la collection")

def Accès_par_dic(num_Doc):
  if num_Doc in index: 
	  return index[num_Doc]
  return {}
 
def traiter_document(var):
    #remplacer les '
    var=var.replace(","," ")
    var=var.replace("."," ")
    var=var.replace("-"," ")
    var=var.replace("!"," ")
    var=var.replace("?"," ")
    var=var.replace(":"," ")
    var=var.replace("="," ")
    var=var.replace("@"," ")
    var=var.replace("&"," ")
    var=var.replace("$"," ")
    var=var.replace("("," ")
    var=var.replace(")"," ")
    var=var.replace("["," ")
    var=var.replace("]"," ")
    var=var.replace("{"," ")
    var=var.replace("}"," ")
    var=var.replace("<"," ")
    var=var.replace(">"," ")
    var=var.replace("'"," ")
    var=var.replace("`"," ")
    var=var.replace("|"," ")
    ponctuation=["\"", "'", "!", "(", ")", "?", ":", "=", ",", ";", ".","`",">","<","}","{","]","[","&","@","|"]

    #decouper le texte en mots et mettre tous les mots en minuscules
    liste_des_mots=word_tokenize(var)
    liste_des_mots = [word.lower() for word in liste_des_mots ]
    
    #supprimer les mots vides
    #print(stopwords.words('english'))
    liste_des_mots = [word for word in liste_des_mots if word not in stopwords.words("english")]

    #supprimer la ponctuation
    liste_des_mots = [word for word in liste_des_mots if word not in ponctuation]

    #supprimer les mots à une seule lettre
    liste_des_mots = [word for word in liste_des_mots if len(word)>1]

    #calcuer la frequence de chaque mot du document et mettre le tout dans un dictionnaire "dict_frequence"
    dict_frequence=dict(collections.Counter(liste_des_mots))

    return(dict_frequence)   

def cleanReq(var):
    var=var.replace(","," ")
    var=var.replace("."," ")
    var=var.replace("-"," ")
    var=var.replace("!"," ")
    var=var.replace("?"," ")
    var=var.replace(":"," ")
    var=var.replace("="," ")
    var=var.replace("@"," ")
    var=var.replace("&"," ")
    var=var.replace("$"," ")
    var=var.replace("("," ")
    var=var.replace(")"," ")
    var=var.replace("["," ")
    var=var.replace("]"," ")
    var=var.replace("{"," ")
    var=var.replace("}"," ")
    var=var.replace("<"," ")
    var=var.replace(">"," ")
    var=var.replace("'"," ")
    var=var.replace("`"," ")
    var=var.replace("|"," ")
    var  = var.lower()
    return var

def codeReq(index):
    dicReq = dict()
    for term in index:
      freq = index[term]
      maximum = sorted(list(index.values()), reverse=True)[0]
      dicReq[term] = float("{:.3f}".format(freq/maximum))
    return dicReq

def mesureInterne(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        somme = 0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme)))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

def mesureDice(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format(((2*somme) / (sommedoc+sommereq))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

def mesureCosinus(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme / math.sqrt(sommedoc*sommereq))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

def mesureJacard(dic_req,reverse_dic_pondere,nbDocument):
    list_doc = dict()
    #construire une liste des documents selectionnée
    dic_doc = dict()
    for iddoc in range(1,(nbDocument+1)):
        #initialiser les somme
        somme, sommedoc, sommereq = 0,0,0
        for word in dic_req:
            #calculer la somme des produits interne 
            if (word,str(iddoc)) in reverse_dic_pondere:
                somme = somme + dic_req[word] * reverse_dic_pondere[(word,str(iddoc))]
                sommedoc = sommedoc + pow(reverse_dic_pondere[(word,str(iddoc))],2)
                sommereq = sommereq + pow(dic_req[word],2)
        #Ajouter le documents a la liste si la somme est différente de zero
        if somme != 0:
            somme = float("{:.3f}".format((somme / (sommedoc+sommereq-somme))))
            dic_doc[str(iddoc)] = somme
    #triée la liste par ordre decroissant
    return dict(sorted(dic_doc.items(), key=lambda item: item[1],reverse=True))

def Modèle_Vectoriel(req,reverse_dic_pondere,nbDocument,methode):
    if methode == 1:
        return(mesureInterne(req,reverse_dic_pondere,nbDocument))
    elif methode == 2:
        return(mesureDice(req,reverse_dic_pondere,nbDocument))
    elif methode == 3:
        return(mesureCosinus(req,reverse_dic_pondere,nbDocument))
    elif methode == 4:
        return(mesureJacard(req,reverse_dic_pondere,nbDocument))

#############################################fichier pickle dans le dossier pickle############################        
output = open("./pickle/index",'rb')
index = pickle.load(output)
infile3 = open("./pickle/Fichier_inverse_pondere",'rb')
reverse_dic_pondere= pickle.load(infile3) 


############################################main####################################


menu = int(input("1-Fonction d'acces 2- modele de recherche"))
if menu == 1:
  acces = int(input("Est ce que vous voulez tester les fonctions d'acces? (1 acces par doc et 2 acces par mot "))
  if acces == 2:
    mot = input("Donner un mot: ")
    start_time = time.time()
    Rechrcher_terme(mot)
    interval = time.time() - start_time  
    print ("Temps de réponse  %.02f" % (interval)," secondes"  )
  else:
    num = input("Donner un numero de document: ")
    start_time = time.time()
    dic = Accès_par_dic(num)
    if dic:
      print(dic)
    else:
      print("Le document n'existe pas")
    interval = time.time() - start_time  
    print ("Temps de réponse  %.02f" % (interval)," secondes"  )
else:
  model = int(input("Quelle modele voulez vous tester? (1 pour model booleen et 2 pour model vectoriel"))
  requete = input('Veuillez saisir votre requete: ')
  if(model == 1):
    start_time = time.time()
    reponse = Modèle_Boolean(requete,index)
    print(reponse)
    interval = time.time() - start_time  
    print ("Temps de réponse du modele booleen  %.02f" % (interval)," secondes"  )
  else:
    #tester le modèle vectoriel
    indexreq=traiter_document(requete)
    dicreq=codeReq(indexreq)
    nbDoc = 3204
    start_vect_time = time.time()
    reponse=Modèle_Vectoriel(dicreq,reverse_dic_pondere,nbDoc,2)
    print(reponse)
    intervalvect = time.time() - start_vect_time  
    print ("Temps de réponse du modele vectoriel  %.02f" % (intervalvect)," secondes"  )

1-Fonction d'acces 2- modele de recherche2
Quelle modele voulez vous tester? (1 pour model booleen et 2 pour model vectoriel2
Veuillez saisir votre requete: comunication language software algebraic preliminary
{'2151': 1.0, '2003': 0.999, '93': 0.998, '1394': 0.998, '1975': 0.998, '2167': 0.998, '2170': 0.998, '3031': 0.998, '3202': 0.998, '1253': 0.996, '825': 0.995, '205': 0.994, '224': 0.994, '655': 0.994, '679': 0.994, '1162': 0.994, '1180': 0.994, '1306': 0.994, '1348': 0.994, '1421': 0.994, '1454': 0.994, '1485': 0.994, '1496': 0.994, '1541': 0.994, '1614': 0.994, '1693': 0.994, '1703': 0.994, '1767': 0.994, '1787': 0.994, '1788': 0.994, '1835': 0.994, '1860': 0.994, '1861': 0.994, '1923': 0.994, '2059': 0.994, '2084': 0.994, '2091': 0.994, '2126': 0.994, '2127': 0.994, '2154': 0.994, '2178': 0.994, '2230': 0.994, '2389': 0.994, '2546': 0.994, '2647': 0.994, '2652': 0.994, '2704': 0.994, '2705': 0.994, '2730': 0.994, '2739': 0.994, '2782': 0.994, '2785': 0.994, '2786': 0.994, '27

**Merci pour votre lecture**